## Functions for plotting metrics for comparison (text in Polish)

> Applies to local experiments not described in the code or documentation.

### Loss plots

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

def plot_loss_same_net():
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=1.2)
    plt.rcParams.update({
        "figure.figsize": (8,6),
        "figure.dpi": 300,
        "axes.linewidth": 1.0,
        "grid.linestyle": "--",
        "grid.alpha": 0.7,
        "legend.frameon": False,
        "legend.loc": "best"
    })

    research_dir = "..\\outputs\\research\\hqnn_parallel\\qreps"
    research_pl = "Głębokość ansatzu"
    metric = "loss"
    metric_pl = "Koszt"
    stage = "val"


    pattern = os.path.join(research_dir, f"[1-9]_{stage}_*.csv")
    file_paths = sorted(glob.glob(pattern), key=lambda x: int(os.path.basename(x).split("_",1)[0]))
    if not file_paths:
        raise Exception("Where?")

    data = {fp: pd.read_csv(fp) for fp in file_paths}

    fig, ax = plt.subplots()
    for fp, df in data.items():
        label = f"{research_pl}: {os.path.basename(fp).split('_',1)[0]}"
        ax.plot(df["epoch"], df[metric], label=label, linewidth=1.5, marker="o", markersize=3)
    ax.set_xlabel("Liczba epok")

    ax.set_ylabel(metric_pl)
    ax.tick_params(pad=5)
    ax.legend()

    axins = inset_axes(ax, width="30%", height="30%", loc="center", borderpad=2)
    axins.patch.set_facecolor("white")
    axins.set_zorder(2)
    for fp, df in data.items():
        x = df["epoch"].values[-5:]
        y = df[metric].values[-5:]
        axins.plot(x, y, linewidth=1.5, marker="o", markersize=3)

    x0 = min(df["epoch"].values[-5:].min() for df in data.values())
    x1 = max(df["epoch"].values[-5:].max() for df in data.values())
    y0 = min(df[metric].values[-5:].min() for df in data.values()) - 0.001
    y1 = max(df[metric].values[-5:].max() for df in data.values()) + 0.001

    axins.set_xlim(x0, x1)
    axins.set_ylim(y0, y1)
    axins.tick_params(labelsize=8)

    mark_inset(ax, axins, loc1=3, loc2=1, fc="none", ec="0.5", linestyle="--", linewidth=0.5, zorder=1)
    plt.tight_layout()
    fig.savefig(os.path.join(research_dir, f"{metric}_{stage}_comparison.png"))
    
plot_loss_same_net()

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

def plot_loss_many_nets():
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=1.2)
    plt.rcParams.update({
        "figure.figsize": (8,6),
        "figure.dpi": 300,
        "axes.linewidth": 1.0,
        "grid.linestyle": "--",
        "grid.alpha": 0.7,
        "legend.frameon": False,
        "legend.loc": "best"
    })

    research_dir = "..\\outputs\\research\\hqnn_parallel\\qc"
    metric = "loss"
    metric_pl = "Koszt"
    stage = "val"


    pattern = os.path.join(research_dir, f"*_{stage}_*.csv")
    file_paths = sorted(glob.glob(pattern), key=lambda x: os.path.basename(x).split("_",1))
    if not file_paths:
        raise Exception("Where?")

    data = {fp: pd.read_csv(fp) for fp in file_paths}

    fig, ax = plt.subplots()
    for fp, df in data.items():
        label = f"{os.path.basename(fp).split('_',1)[0]}"
        ax.plot(df["epoch"], df[metric], label=label, linewidth=1.5, marker="o", markersize=3)
    ax.set_xlabel("Liczba epok")

    ax.set_ylabel(metric_pl)
    ax.tick_params(pad=5)
    ax.legend()

    axins = inset_axes(ax, width="30%", height="30%", loc="center", borderpad=2)
    axins.patch.set_facecolor("white")
    axins.set_zorder(2)
    for fp, df in data.items():
        x = df["epoch"].values[-5:]
        y = df[metric].values[-5:]
        axins.plot(x, y, linewidth=1.5, marker="o", markersize=3)

    x0 = min(df["epoch"].values[-5:].min() for df in data.values())
    x1 = max(df["epoch"].values[-5:].max() for df in data.values())
    y0 = min(df[metric].values[-5:].min() for df in data.values()) - 0.001
    y1 = max(df[metric].values[-5:].max() for df in data.values()) + 0.001

    axins.set_xlim(x0, x1)
    axins.set_ylim(y0, y1)
    axins.tick_params(labelsize=8)

    mark_inset(ax, axins, loc1=3, loc2=1, fc="none", ec="0.5", linestyle="--", linewidth=0.5, zorder=1)
    plt.tight_layout()
    fig.savefig(os.path.join(research_dir, f"{metric}_{stage}_comparison.png"))
    
plot_loss_many_nets()

### Accuracy plots

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

def plot_loss():
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=1.2)
    plt.rcParams.update({
        "figure.figsize": (8,6),
        "figure.dpi": 300,
        "axes.linewidth": 1.0,
        "grid.linestyle": "--",
        "grid.alpha": 0.7,
        "legend.frameon": False,
        "legend.loc": "best"
    })

    research_dir = "..\\outputs\\research\\hqnn_parallel\\qc"
    metric = "accuracy"
    metric_pl = "Dokładność [%]"
    stage = "train"

    pattern = os.path.join(research_dir, f"*_{stage}_*.csv")
    file_paths = sorted(glob.glob(pattern), key=lambda x: os.path.basename(x).split("_",1))
    if not file_paths:
        raise Exception("Where?")

    data = {fp: pd.read_csv(fp) for fp in file_paths}

    fig, ax = plt.subplots()
    for fp, df in data.items():
        df[metric] *= 100.0 # %
        label = f"{os.path.basename(fp).split('_',1)[0]}"
        ax.plot(df["epoch"], df[metric], label=label, linewidth=1.5, marker="o", markersize=3)
    ax.set_xlabel("Liczba epok")

    ax.set_ylabel(metric_pl)
    ax.tick_params(pad=5)
    ax.legend(loc="lower right")

    axins = inset_axes(ax, width="30%", height="30%", loc="center", borderpad=2)
    axins.patch.set_facecolor("white")
    axins.set_zorder(2)
    for fp, df in data.items():
        x = df["epoch"].values[-5:]
        y = df[metric].values[-5:]
        axins.plot(x, y, linewidth=1.5, marker="o", markersize=3)

    x0 = min(df["epoch"].values[-5:].min() for df in data.values())
    x1 = max(df["epoch"].values[-5:].max() for df in data.values())
    y0 = min(df[metric].values[-5:].min() for df in data.values()) - 0.001
    y1 = max(df[metric].values[-5:].max() for df in data.values()) + 0.001

    axins.set_xlim(x0, x1)
    axins.set_ylim(y0, y1)
    axins.tick_params(labelsize=8)
    # axins.yaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5", linestyle="--", linewidth=0.5, zorder=1)
    plt.tight_layout()
    fig.savefig(os.path.join(research_dir, f"{metric}_{stage}_comparison.png"))
    
plot_loss()

In [ ]:
import sys
from pathlib import Path

try:
    here = Path(__file__).resolve().parent
except NameError:
    here = Path.cwd()
if here.name != "src":
    src_dir = here.parent / "src"
else:
    src_dir = here
if not src_dir.is_dir():
    raise ImportError(f"No 'src' in {src_dir}")
sys.path.insert(0, str(src_dir))

#################################

import torch
import pennylane as qml
from pennylane import transforms

from models.hqnn_quanv.hqnn_quanv import Quanv

num_qubits = 4
num_qreps = 1
layer = Quanv(1, 2, 1, 2, num_qubits, num_qreps)

decomposed_circuit = transforms.decompose(layer._circuit)  

weight_shapes = {"weights": (num_qreps, num_qubits, 3)}
dummy_inputs  = torch.zeros(num_qubits)
dummy_weights = torch.zeros((num_qreps, num_qubits, 3))

qml.drawer.use_style("black_white")
fig, ax = qml.draw_mpl(decomposed_circuit)(dummy_inputs, dummy_weights)

In [ ]:
import sys
from pathlib import Path

try:
    here = Path(__file__).resolve().parent
except NameError:
    here = Path.cwd()
if here.name != "src":
    src_dir = here.parent / "src"
else:
    src_dir = here
if not src_dir.is_dir():
    raise ImportError(f"No 'src' in {src_dir}")
sys.path.insert(0, str(src_dir))

#################################

import torch
import pennylane as qml
from pennylane import transforms

from models.hcqtcnn.hcqtcnn import QLayer

num_qubits = 4
num_qreps = 1
layer = QLayer(num_qubits, num_qreps)

decomposed_circuit = transforms.decompose(layer._circuit)  

weight_shapes = {"weights": (num_qreps, num_qubits, 3)}
dummy_inputs  = torch.zeros((1, num_qubits))
dummy_weights = torch.zeros((num_qreps, num_qubits, 3))

qml.drawer.use_style("black_white")
fig, ax = qml.draw_mpl(decomposed_circuit)(dummy_inputs, dummy_weights)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

categories = [1, 3, 5, 7, 9]
values     = [98.10, 98.87, 99.19, 99.07, 99.06]

sns.set_style("whitegrid")
plt.rcParams.update({
    "figure.figsize": (8,6),
    "figure.dpi": 300,
    "axes.linewidth": 1.0,
    "grid.linestyle": "--",
    "grid.alpha": 0.7,
    "legend.frameon": False,
    "legend.loc": "best"
})

plt.figure(figsize=(8, 6), dpi=300)

def fmt(x):
    if float(x).is_integer():
        return f"{int(x)}"
    s = f"{x:.2f}".replace('.', ',')
    return s

colors = sns.color_palette("tab10", len(categories))

bars = plt.bar(categories, values, color=colors, edgecolor="black", width=0.8)

for bar in bars:
    h = bar.get_height()
    plt.annotate(fmt(h),
                 xy=(bar.get_x() + bar.get_width() / 2, h),
                 xytext=(0, 5), textcoords="offset points",
                 ha="center", va="bottom", fontsize=17)

ax = plt.gca()
ax.set_xlabel("Liczba kubitów", labelpad=10)
ax.set_ylabel("Dokładność [%]", labelpad=10)

ax.set_xticks(categories)
ax.set_xlim(min(categories) - 1, max(categories) + 1)

ymin, ymax = min(values) - 0.1, max(values) + 0.22
ax.set_ylim(ymin, ymax)

ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: fmt(x)))

sns.despine(trim=True)
plt.tight_layout()
plt.savefig("hqnn-parallel_qubits.png", dpi=300)
plt.show()